# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: MLOps**

### Tecnológico de Monterrey

## **Fase 1 - Avance de proyecto - Equipo 7**
### **Problema de asignación de créditos: South German Dataset.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## **Manipulación y preparación de datos**

## **Exploración y preprocesamiento de datos**

## **Versionado de datos**

## **Construcción, ajuste y evaluación de Modelos de Machine Learning**

## **Aplicación de Mejores Prácticas en el Pipeline de Modelado**

## **Estructuración y Refactorización del Código**